# Project 2
Mitchell Morrison and Christian Gould

# Part One
We have decided to use the COVID 19 dataset in pair with a dataset of holidays and their dates around the world. We plan on enriching the COVID dataset by showing the relationship between the period following a holiday and the level of covid in the country celebrating. The holidays dataset was gathered for the purpose of testing it in conjunction with covid data. 



Dataset links:
1. https://www.kaggle.com/sandhyakrishnan02/latest-covid-19-dataset-worldwide
2. https://www.kaggle.com/vbmokin/covid19-holidays-of-countries?select=holidays_df_of_70_countries_for_covid_19.csv

In [2]:
import numpy as np
import pandas as pd

## Understanding our datasets

### Holidays dataset

In [3]:
holidays = pd.read_csv('datasets/holidays_df_of_70_countries_for_covid_19.csv')
holiday_columns_of_interest = ['country', 'ds', 'ds_holidays']
holidays = holidays[holiday_columns_of_interest]
holidays.head()

,country,ds,ds_holidays
0,Argentina,2020-03-02,2020-02-24
1,Argentina,2020-03-03,2020-02-25
2,Argentina,2020-03-31,2020-03-24
3,Argentina,2020-04-16,2020-04-09
4,Argentina,2020-04-17,2020-04-10


In [4]:
holidays.columns

Index(['country', 'ds', 'ds_holidays'], dtype='object')

What are our variables from the holiday dataset? 

* ds_holidays - date of the holiday
  * date format - YYYY-MM-DD
* holiday - name of the holiday
  * ex. Christmas
* ds - ds_holidays plus time delta of 7 days
* country - country of holiday
* code - conutry abbreviation
* country_official_name - formal country name
* lower_window - ds minus time delta of 3 days (early COVID onset boundary)
* upper_window - ds plus time delta of 3 days (late COVID onset boundary)
* prior_scale -
* source - where holiday data is retrieved from

Of these, we are only interested in: *country, ds, ds_holidays*
* This is because we are only interested in the dates of the holidays and the country that the holiday is in. ds is not necessary, but is helpful in order for us to check dates that are a week away from the holiday.

Here is the range of the dates in the holiday set:


In [7]:
print("Latest date: ", max(holidays['ds']))
print("Earliest date: ", min(holidays['ds']))
print("number of counrties: ", len(holidays['country'].unique()))

# print the rows with missing values
print("number of rows with missing values: ", holidays.isnull().sum().sum())


max date:  2021-01-07
min date:  2020-01-28
number of counrties:  70
number of rows with missing values:  0


### COVID 19 dataset

In [5]:
covid = pd.read_csv('datasets/owid-covid-data.csv')
covid.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


Wow, this dataset has a ton of columns!! Lets narrow it down to only columns that are related to transmission and testing results. <br>
These columns are:
<list>
    <li> iso_code - country code
    <li> continent - exactly what you think
    <li> location - country name
    <li> date - date of record for instance
    <li> total_cases - total covid cases for the country
    <li> new_cases - new cases this day
    <li> new_cases_smoothed - new cases smoothed over XXX day period
    <li> total_deaths - total deaths for the country
    <li> new_deaths - new deaths this day
    <li> total_cases_per_million - ratio of total cases to million of population
    <li> new_cases_per_million - ratio of new cases today to million of population
    <li> new_cases_smoothed_per_million - ratio of new cases smoothed over XXX day period to million of population
    <li> reproduction_rate - real time estimate of transmission factor of covid

In [6]:
columnsOfInterest = ['iso_code', 'location', 'date', 'total_cases', 'new_cases',
        'new_cases_smoothed', 'total_deaths', 'new_deaths', 'total_cases_per_million',
        'new_cases_per_million', 'new_cases_smoothed_per_million']
covid[columnsOfInterest]

,iso_code,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million
0,AFG,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,0.126,0.126,NaN
1,AFG,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,0.126,0.000,NaN
2,AFG,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,0.126,0.000,NaN
3,AFG,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,0.126,0.000,NaN
4,AFG,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,0.126,0.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
153167,ZWE,Zimbabwe,2022-01-04,217678.0,1591.0,1447.143,5078.0,31.0,14423.240,105.419,95.887
153168,ZWE,Zimbabwe,2022-01-05,219057.0,1379.0,1644.143,5092.0,14.0,14514.612,91.372,108.940
153169,ZWE,Zimbabwe,2022-01-06,220178.0,1121.0,1207.143,5108.0,16.0,14588.889,74.277,79.985
153170,ZWE,Zimbabwe,2022-01-07,221282.0,1104.0,1146.286,5136.0,28.0,14662.039,73.151,75.952


## Merging our datasets
### We will merge these two dataframes using a left join on the COVID table with the country and date attributes
It may also work to use an inner join to only keep days that are holidays and the window of transmission that we are interested in

In [56]:
new_df = pd.merge(covid, holidays,  how='left', left_on=['location','date'], right_on = ['country','ds_holidays'])


In [57]:
new_df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,ds_holidays,holiday,ds,country,code,country_official_name,lower_window,upper_window,prior_scale,source
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
